# Correctness Test

## Set up

In [61]:
from neo4j import GraphDatabase
import pandas as pd


credentialsNeo4j = pd.read_json('credentialsNeo4j.json')

use_local = False
location = "local"

uri = "neo4j://localhost:"

if use_local == True:
    location = 'local'
    port     = 7687
else:
    location = 'server'
    port     = 7688
    
username = credentialsNeo4j[location][0]['user']
password = credentialsNeo4j[location][0]['pwd']
#print(username, password)
driver = GraphDatabase.driver(uri + str(port), auth=(username, password))
session = driver.session()

In [62]:
## parameter settings

# original set
b = 'Block'
t = 'Transaction'
a = 'Address'

# test set
bt = b + '_test'
tt = t + '_test'
at = a + '_test'

In [63]:
##  test settings
height=684708
txid = '7b8f901f2244a3dcc9d2757cefe660849d8800a964fda944922df06450468153'
blockClass = b
txClass = t
addrClass = a

## Analysis

### Functions

#### Queries

In [52]:
def getBlockPropertiesByHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}}) RETURN b'#.format(blockClass, height)

def getBlockEdgesByHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}})<-[r]-() RETURN r'

def getTransactionsFromBlockHeightQuery(blockClass, height):
    return f'MATCH (b:{blockClass} {{height: {height}}})<-[:BELONGS_TO]-(t) RETURN t.txid'

def getTransactionPropertiesByTxidQuery(txClass, txid):
    return f'MATCH (t:{txClass} {{txid: \"{txid}\"}}) RETURN t'

def getAddressFromTransactionIDQuery(txClass, txid, addrClass):
    return f'MATCH (t:{txClass} {{txid: \"{txid}\"}})-[]-(a:{addrClass}) RETURN a'
    
def getAddressPropertiesByAddressQuery(addrClass, address):
    return f'MATCH (a:{addrClass} {{address: \"{address}\"}}) RETURN a'



#### Other functions

In [56]:
def runQuery(query):
    return session.run(query).data()

### Correctnes check

### Testing

In [65]:
session.run("Create (t:Test {name: 'Test'})")

In [16]:
getBlockEdgesByHeightQuery(b, height)
#getBlockPropertiesByHeightQuery(blockClass, height)

'MATCH (b:Block {height: 684708})<-[r]-() RETURN r'

In [30]:
def getProperties(tx, height):
    tx.run(query_getBlockProperties, height)
#           height         = event["txid"], 
    
#with driver.session() as session:
#    session.write_transaction(getBlockProperties, height)

In [25]:
result = session.run("MATCH (b:Block) RETURN b")

In [26]:
result

In [24]:
result.data()

[]

In [64]:
result = session.run(getBlockPropertiesByHeightQuery(blockClass, 1)).data()
result

[{'b': {'blockDate': neo4j.time.Date(2009, 1, 9),
   'mediantime': neo4j.time.DateTime(2009, 1, 9, 2, 54, 0.0, tzinfo=<UTC>),
   'hash': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048',
   'height': 1}}]

In [37]:
result = session.run(getBlockEdgesByHeightQuery(b, height)).data()
len(result)#.data()

1570

In [34]:
result = session.run(getTransactionsFromBlockHeightQuery(blockClass, height)).data()
result

[{'t.txid': 'e5602c44d2dc4e83d62286e3777847d3f733cdcf2fba26b83cda26e0880d7a01'},
 {'t.txid': 'b64d905e8733a265483bcb1f2694d5ccfaa17f85e2ee2199c4470fdd171aacf0'},
 {'t.txid': '50f2c203f18236be46185482a8895a26e23e42cede0bfa0fd2fddba99d7f06ad'},
 {'t.txid': '5052b7d8a059d5077142fa68a203f81f77ffe15627f51e34e4c62e0e6caa788a'},
 {'t.txid': '2d140a60ac9a546b4cfb72d1056ac581ab26b1ed2a9889c7ea47a0e2e8bd1c2d'},
 {'t.txid': '66c57bcae9b51ed5705ec3b6467315c23b591371aba0c5c68dc1b0b6fb5a897c'},
 {'t.txid': 'c1425bc4726cc00d9f1fdd4a0105d21177fea51af37b251feae1d0c67db01a94'},
 {'t.txid': '797dcf6f4259571484c3b0d155aa26f4b17196eaf97b7c11d14a93e8b03f550e'},
 {'t.txid': '7d24f5934c699fbb67000d73a0bc476a67a83c7b9cdc20667c283f63233bddc0'},
 {'t.txid': '58882d3b8436be78141c05f7bc7c5e78bc34c9a8eefec02dc515133c16298693'},
 {'t.txid': '54d1179968f3ab4a25827b2aea296fbb3fd0bbc958315abd7d740b620e3c712e'},
 {'t.txid': '8f810cce34df8b20baa43940c98d8bae3be9bea9fadae89862b3c6abc3d30778'},
 {'t.txid': '8075b265c908430

In [53]:
query = getAddressFromTransactionIDQuery(txClass, txid, addrClass)
print(query)
result = session.run(query).data()
result

MATCH (t:Transaction {txid: "7b8f901f2244a3dcc9d2757cefe660849d8800a964fda944922df06450468153"})-[]-(a:Address) RETURN a


[{'a': {'inDegree': 1,
   'address': '1Gpbp7bbsj3d4m4k18r9VMnWJKPFBRqN19',
   'outDegree': 0}},
 {'a': {'inDegree': 0,
   'address': '3KLHctsA3VXhj4498Q3YJEVByyTuNV8zz5',
   'outDegree': 1}},
 {'a': {'inDegree': 1,
   'address': '386DCYe5Av1jqXvP1vTw38awLsqrcvMPXq',
   'outDegree': 0}}]

In [57]:
runQuery(getAddressFromTransactionIDQuery(txClass, txid, addrClass))

[{'a': {'inDegree': 1,
   'address': '1Gpbp7bbsj3d4m4k18r9VMnWJKPFBRqN19',
   'outDegree': 0}},
 {'a': {'inDegree': 0,
   'address': '3KLHctsA3VXhj4498Q3YJEVByyTuNV8zz5',
   'outDegree': 1}},
 {'a': {'inDegree': 1,
   'address': '386DCYe5Av1jqXvP1vTw38awLsqrcvMPXq',
   'outDegree': 0}}]

## End connection

In [ ]:
session.close()
driver.close()